---
# --- Day 16: Ticket Translation ---
---

In [39]:
from itertools import chain
import numpy as np

### Input

In [158]:
with open("data/16_input.txt") as f:
    data = [l.strip() for l in f.readlines()]
with open("data/16_input_test.txt") as f:
    data_test = [l.strip() for l in f.readlines()]
with open("data/16_input_test2.txt") as f:
    data_test2 = [l.strip() for l in f.readlines()]

In [100]:
def combine_ranges(intervals):
    tot_range = range(0, 0)
    for i in intervals:
        extremes = i.split("-")
        tot_range = chain(tot_range, range(int(extremes[0]), int(extremes[1])+1))
    return list(tot_range)

def read_data(data):
    field_values = {}
    empty_line = False
    for i, l in enumerate(data):
        if len(l) == 0:
            break
        else:
            parts = l.split(": ")
            field = parts[0]
            nums = parts[1].split(" or ")
            field_values[field] = combine_ranges(nums)
    
    my_tkt = [int(v) for v in data[i+2].split(",")]
    nearby_tkts = [[int(v) for v in data[p].split(",")] for p in range(i+5, len(data))]
    
    return field_values, my_tkt, nearby_tkts

In [170]:
field_values, my_tkt, nearby_tkts = read_data(data)
field_values_test, my_tkt_test, nearby_tkts_test = read_data(data_test)
field_values_test2, my_tkt_test2, nearby_tkts_test2 = read_data(data_test2)

### Part 1: check validity

In [108]:
def check_validity(tkts, dict_fields):
    all_field_values = list(dict_fields.values())
    mask = np.array([[np.any([v in fv for fv in all_field_values]) for v in t] for t in tkts])
    tkt_validity = np.all(mask, axis=1)
    all_invalid_values = np.array(tkts)[~mask]
    return tkt_validity, all_invalid_values

In [110]:
validity_test, all_invalid_values_test = check_validity(nearby_tkts_test, field_values_test)
print(validity)
print(sum(all_invalid_values.flatten()))

[ True False False False]
71


In [111]:
validity, all_invalid_values = check_validity(nearby_tkts, field_values)
print(validity)
print(sum(all_invalid_values.flatten()))

[ True  True  True False  True  True  True  True  True False  True  True
  True  True  True  True  True  True  True  True False  True  True  True
  True False  True  True  True  True False  True  True  True  True  True
 False False  True  True  True  True  True  True  True  True  True  True
  True False  True  True  True  True  True  True  True  True False  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True False  True  True False False False False  True
 False False  True  True  True False  True  True  True  True  True  True
  True False  True False  True False  True  True False  True  True  True
  True False  True  True  True False False  True False  True  True  True
 False  True  True  True  True  True  True  True  True  True  True  True
  True False  True  True  True  True False  True  True False  True  True
  True  True  True  True  True  True  True  True  True  True False  True
  True  True  True  True  True False  True  True  T

### Part 2: field correspondances

In [143]:
valid_tkts = np.array(nearby_tkts)[validity,:]
valid_tkts_test = np.array(nearby_tkts_test)[validity_test, :]

In [223]:
def get_field_correspondance(tkts, dict_fields):
    all_field_values = list(dict_fields.values())
    mask3d = np.array([[[v in fv for fv in all_field_values] for v in t] for t in tkts])
    c = np.all(mask3d, axis=0)
    sum_good = sum(c)
    field_order = -1*np.ones(len(all_field_values))
    pos_in_list = np.arange(len(all_field_values))
    pos_in_tkt = np.arange(len(all_field_values))
    while ((1 in sum_good)) and (len(c) > 1):
        idx_list = np.where(sum_good==1)[0][0]
        idx_tkt = np.where(c[:, idx_list])[0][0]
        field_order[pos_in_tkt[idx_tkt]] = pos_in_list[idx_list]
        pos_in_tkt = np.delete(pos_in_tkt, idx_tkt)
        pos_in_list = np.delete(pos_in_list, idx_list)
        c = np.delete(np.delete(c, idx_tkt, 0), idx_list, 1)
        sum_good = sum(c)
    field_order[pos_in_tkt[0]] = pos_in_list[0]
    field_order = [int(o) for o in field_order]
    if not (-1 in field_order):
        print(f"Order found: {field_order}.")
    else:
        print("No solution found!")
    return field_order

In [225]:
 field_order = get_field_correspondance(nearby_tkts_test2, field_values_test2)

Order found: [1, 0, 2].


In [226]:
 field_order = get_field_correspondance(valid_tkts, field_values)

Order found: [10, 3, 0, 8, 7, 19, 5, 12, 15, 14, 17, 16, 18, 13, 1, 9, 2, 4, 6, 11].


#### Multiplication of the ticket values corresponding to the first 6 fields

In [237]:
departure_fields = [f in range(6) for f in field_order]

In [240]:
np.prod(np.array(my_tkt)[departure_fields])

1307550234719